In [2]:
import numpy as np
from stl import mesh
import plotly.graph_objects as go
from plotly.colors import sequential

def slice_stl_3d(file_path, layer_height):
    # Load STL file
    stl_mesh = mesh.Mesh.from_file(file_path)
    facets = stl_mesh.vectors  # Extract triangle vertices
    
    # Determine Z bounds
    z_min = np.min(facets[:, :, 2])
    z_max = np.max(facets[:, :, 2])
    
    # Generate slicing planes
    z_levels = np.arange(z_min, z_max, layer_height)
    
    # Store slices
    slices = []
    for z in z_levels:
        layer_lines = []
        for triangle in facets:
            intersections = intersect_plane_with_triangle(z, triangle)
            if intersections is not None:
                layer_lines.append(intersections)
        slices.append((z, layer_lines))
    
    return slices

def intersect_plane_with_triangle(z, triangle):
    """Find intersections of a horizontal plane with a triangle."""
    edges = [(triangle[i], triangle[(i + 1) % 3]) for i in range(3)]
    points = []
    for p1, p2 in edges:
        if (p1[2] <= z and p2[2] >= z) or (p1[2] >= z and p2[2] <= z):
            t = (z - p1[2]) / (p2[2] - p1[2]) if p1[2] != p2[2] else 0
            intersect_point = p1 + t * (p2 - p1)
            points.append(intersect_point)
    if len(points) == 2:
        return points
    return None


def visualise_slices_3d(slices):
    fig = go.Figure()

    # Generate colours using Plotly's Viridis colour scale
    colour_scale = sequential.Viridis  # Plotly's Viridis colour scale
    colours = [colour_scale[int(i * len(colour_scale) / len(slices))] for i in range(len(slices))]

    for (z, lines), colour in zip(slices, colours):
        for line in lines:
            x = [line[0][0], line[1][0]]  # X-coordinates
            y = [line[0][1], line[1][1]]  # Y-coordinates
            z_coords = [z, z]  # Fixed Z level
            fig.add_trace(go.Scatter3d(
                x=x, y=y, z=z_coords,
                mode='lines',
                line=dict(color=colour, width=2)
            ))

    # Update layout for 3D visualisation
    fig.update_layout(
        scene=dict(
            xaxis_title="X Axis",
            yaxis_title="Y Axis",
            zaxis_title="Z Axis",
        ),
        title="3D Sliced Layers",
        margin=dict(l=0, r=0, b=0, t=40),
    )

    # Show interactive plot
    fig.show()


# Usage example
file_path = "/home/a/Documents/3rd_Year/3YP/Code/attempt2/MN.stl"
layer_height = 300  # Example layer height
slices = slice_stl_3d(file_path, layer_height)
visualise_slices_3d(slices)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed